In [1]:
##########==========##########==========##########==========##########==========

# H - Header

#### H1 - Libraries

In [2]:
## standard foundational libraries
import numpy             as np
import pandas            as pd
import matplotlib.pyplot as plt

## import specific functions
from matplotlib.colors  import hsv_to_rgb, to_hex
from os                 import mkdir, listdir
from os.path            import isfile, isdir
from datetime           import datetime, timedelta
from cartopy            import feature
from cartopy.crs        import LambertConformal, PlateCarree
from cartopy.io import shapereader
from cartopy.feature import ShapelyFeature
from dbfread            import DBF
from docx               import Document
from textwrap           import fill as txt_wrap
from geopy.distance     import geodesic
from ipyparallel        import Cluster
from sklearn.cluster    import AgglomerativeClustering
from functools          import partial
from scipy.spatial      import Voronoi
from shapely.geometry   import Polygon
from shapely.ops        import unary_union
from shapely.validation import make_valid

#### H2 - Basic Automation

In [3]:
## set up standard directories if needed
def make_standard_file_system():
    for i in ['A_Input', 'B_Intermediate', 'C_Output']:
        if not isdir(i): mkdir(i)

## log time elapsed
time_log = dict()
def log_time(the_id = 'End Log'):
    
    ## construct new time stamp
    now_time = str(datetime.now().hour).zfill(2)
    now_time = now_time +':'+ str(datetime.now().minute).zfill(2)
    now_time = now_time +':'+ str(datetime.now().second).zfill(2)

    ## add to time log
    if the_id == 'End Log':
        time_log['End'] = now_time
        print('Time log:')
        for i in time_log.keys():
            print(i.rjust(5) + ':', time_log[i])
    else:
        time_log[the_id] = now_time
        
## toggle cache versus build
def build_or_cache(function, address, permit):
    if permit and isfile(address):
        print('Build/Cache Decision: Cache')
        the_file = pd.read_csv(address, index_col = 0)
    else:
        print('Build/Cache Decision: Build')
        the_file = function()
        the_file.to_csv(address)
    return the_file
    
## execute functions
make_standard_file_system()
log_time('H2')

#### H3 - Settings

In [4]:
## set color palette
set_color = {
    'AzureDark'    :(7/12, 1.0, 0.4),
    'AzureMedium'  :(7/12, 0.7, 0.7),
    'AzureLight'   :(7/12, 0.4, 1.0),
    'AzureBG'      :(7/12, 0.1, 1.0),
    
    'OrangeDark'   :(1/12, 1.0, 0.4),
    'OrangeMedium' :(1/12, 0.7, 0.7),
    'OrangeLight'  :(1/12, 0.4, 1.0),
    'OrangeBG'     :(1/12, 0.1, 1.0)
    }
for i in set_color.keys(): set_color[i] = to_hex(hsv_to_rgb(set_color[i]))

## set font sizes
set_font = {
    'small' : 15,
    'medium': 20,
    'large' : 25
    }

## time-saver settings
set_acceleration = {
    'dbf_cache'              : True,
    'sample_size'            : 1/5, ## 1/5, 1/10, 1/20
    'pop_cache'              : True,
    'distance_parallel_cores': 8,
    'distance_cache'         : True,
    'cluster_parallel_cores' : 8,
    'cluster_L1_cache'       : True,
    'cluster_L2_cache'       : True
    }

## map parameters
set_map = {
    'bounds'  : [-124.73 + 5, -66.95 - 5, 25.12 - 3.3, 49.38 + 3.3],
    'map_proj': LambertConformal(
        central_longitude = (-124.73 - 66.95) / 2,
        central_latitude = (25.12 + 49.38) / 2,
        standard_parallels = (25.12, 49.38))
    }
log_time('H3')

In [5]:
## -- variable codes codes
## DP02_0068E - Bachlor's Degree* (out of total pop)
## DP03_0088E - Income Per Capita** (compare to average)
## DP05_0001E - Total Population*
## DP05_0077E - Non-Hispanic White
## life_expect - life expectancy** (compare to average)
## rep_vote   - 2020 POTUS republican voters* (out of total votes out of pop)
## total_vote - 2020 POTUS total voters*

# GD - Gather Data / RD - Refine Data

#### GD1 - read census tract geographic data

In [6]:
def read_geo_data(directory = 'A_Input/tracts_dbf'):
    
    ## list dbf files in target directory
    dbf_addr = listdir(directory)
    dbf_addr = [i for i in dbf_addr if i[-3::] == 'dbf']
    
    ## define relevant columns from each
    desired_columns = {'GEOID': str,
                       'STATEFP': str, 'COUNTYFP':str, 'TRACTCE':str,
                        'INTPTLAT': float, 'INTPTLON': float, 'ALAND': int}
    
    ## read in all dbf files
    dbf_data = []
    for i in dbf_addr:
        i_dbf = pd.DataFrame(iter(DBF(directory + '/' + i)))
        i_dbf = i_dbf[desired_columns.keys()].astype(desired_columns)
        dbf_data.append(i_dbf)
        
    ## compile data into a single file
    dbf_data = pd.concat(dbf_data, axis = 0).sort_values('GEOID')
    dbf_data['count'] = 1
    dbf_data = dbf_data.reset_index(drop = True)
    
    ## repair GEOID irregularities
    dbf_data['GEOID'] = dbf_data['GEOID'].astype(str).str.zfill(11)
    
    ## export data
    return dbf_data

## execute code
geo_data = build_or_cache(function = read_geo_data,
                          address = 'B_Intermediate/dbf_data.csv.gz',
                          permit = set_acceleration['dbf_cache'])
log_time('GD1')

Build/Cache Decision: Cache


#### RD1 - draw a sample from the census tract geographic data and exclude outlier tracts

In [7]:
def refine_geo_data(dat = geo_data, too_rural = 20.720e6 * 5,
                    too_much = set_acceleration['sample_size']):
    
    ## filter out extremely rural areas (< 100 people per square mile)
    dat = dat.loc[dat.ALAND < too_rural, :]
    
    ## take systematic sample of the data
    i = np.arange(0, dat.shape[0]) % int(1 / too_much)
    dat = dat.loc[i == 0, ]
    
    ## set index
    dat = dat.rename({'GEOID':'GEO_ID'}, axis = 1).set_index('GEO_ID')
    dat.index = 'ct' + dat.index.astype(str).str.zfill(11)

    ## return data
    return dat

## execute code
geo_data = refine_geo_data()
log_time('RD1')

#### GD2 - read population data

In [8]:
def read_pop_data(roster = 'A_Input/sources.csv'):
    
    ## read in data file roster
    roster = pd.read_csv(roster).set_index('OBJ_NAME')
    
    ## load all datasets in the roster file
    pop_data = dict()
    for i in roster.index:
        var_names = roster.loc[i, 'VAR_NAME'].split(';')
        pop_data[i] = pd.read_csv('A_Input' + '/' + roster.loc[i, 'FILE_NAME'],
            usecols = var_names, dtype = str)
        
    ## merge census datasets
    census_i = roster.index[roster.SOURCE == 'data.census.gov'].values
    census_dat_count = 0
    
    for i in census_i:
        pop_data[i] = pop_data[i].loc[1::, :].set_index('GEO_ID')
        if census_dat_count < 1:
            pop_data['census'] = pop_data[i]
            pop_data.pop(i)
            census_dat_count += 1
        else:
            pop_data['census'] = pop_data['census'].join(pop_data[i])
            pop_data.pop(i)
            census_dat_count += 1
            
    ## convert census data to numeric
    def robust_int(x):
        try: x = int(x)
        except: x = 0
        return x
    map_robust_int = lambda x: x.map(robust_int)
    pop_data['census'] = pop_data['census'].apply(map_robust_int)

    return pop_data

## execute code - see RD2

#### RD2 - refine and compile population data

In [9]:
def refine_pop_data(pop):
    
    ## -- standardize geographic codes as needed
    state_fips = pop.pop('state_fips')# if needed in future; not currently used
    pop['census'].index = pop['census'].index.str.replace('1400000US', '')

    ## -- refine life expectancy data and merge into census data
    pop['lifespan'].columns = ['GEO_ID', 'life_expect']
    pop['lifespan'].life_expect = pop['lifespan'].life_expect.astype(float)
    pop['census'] = pop['census'].join(pop['lifespan'].set_index('GEO_ID'))
    pop.pop('lifespan')
    
    ## -- refine voting data
    
    ## filter to necessary data
    i = (pop['vote'].party == 'REPUBLICAN') & (pop['vote'].year == '2020')
    pop['vote'] = pop['vote'].loc[i].drop(['year', 'party', 'state_po'],
                                          axis = 1)
    
    ## impute total votes (data is irregular from state to state)
    temp = pop['vote'].copy()
    temp = temp.drop(['totalvotes'], axis = 1)
    temp = temp.set_index(['county_fips', 'mode']).astype(int).reset_index()
    temp = temp.groupby(['county_fips', 'mode']).sum().reset_index()
    total_vote = temp.loc[temp['mode'] == 'TOTAL'].set_index('county_fips')
    total_vote = total_vote.drop('mode', axis = 1)
    seg_vote = temp.loc[temp['mode'] != 'TOTAL'].groupby('county_fips').sum()
    total_vote = pd.concat({'Total': total_vote, 'Alt': seg_vote}, axis = 1)
    total_vote = total_vote.max(axis = 1)
    total_vote = pd.DataFrame({'repvotes':total_vote})
    pop['vote'] = pop['vote'].join(total_vote, on = 'county_fips')
    pop['vote'] = pop['vote'].drop_duplicates('county_fips')
    pop['vote'] = pop['vote'].drop(['mode', 'candidatevotes'], axis = 1)
    del seg_vote, temp, total_vote
    
    ## calculate percentage voting republican
    pop['vote'] = pop['vote'].set_index('county_fips').astype(float)
    pop['vote']['reppct'] = pop['vote']['repvotes'] / pop['vote']['totalvotes']
    
    ## calculate percentage of population that voted
    county_total = pd.DataFrame(pop['census']['DP05_0001E'])
    county_total['county'] = [i[0:5] for i in county_total.index]
    county_total = county_total.groupby('county').sum().astype(int)
    pop['vote'] = pop['vote'].join(county_total)
    pop['vote']['totalpct'] = pop['vote']['totalvotes'] / pop['vote']['DP05_0001E']
    pop['vote'].loc[pop['vote'].totalpct > 1, 'totalpct'] = 159633396 / 331449281
    pop['vote'] = pop['vote'][['reppct', 'totalpct']]
    
    ## merge voting data into census and convert to counts
    pop['census']['county'] = [i[0:5] for i in pop['census'].index]
    pop['census'] = pop['census'].reset_index().set_index('county').join(pop['vote'])
    pop = pop['census'].set_index('GEO_ID')
    pop['state'] = [i[0:2] for i in pop.index]
    
    ## -- impute missing data
    
    ## impute at the state level
    state_mean = pop.copy()[['life_expect', 'reppct', 'totalpct', 'state']]
    state_mean = state_mean.groupby('state').mean().round(2)
    state_mean = state_mean.loc[pop.state]
    for i in state_mean.columns:
        j = pop[i].isna().values
        pop.loc[j, i] = state_mean.loc[j, i].values
    
    ## impute at the national level
    for i in state_mean.columns:
        j = pop[i].isna().values
        pop.loc[j, i] = pop[i].mean()
    del state_mean
    
    ## convert vote proportions to counts
    pop['rep_vote'] = pop['reppct'] * pop['totalpct'] * pop['DP05_0001E']
    pop['rep_vote'] = pop['rep_vote'].round().astype(int)
    pop['total_vote'] = (pop['totalpct'] * pop['DP05_0001E']).round().astype(int)
    pop = pop.drop(['reppct', 'totalpct', 'state'], axis = 1).round(1)
    
    ## add prefix to GEO_ID
    pop.index = 'ct' + pop.index.astype(str).str.zfill(11)
    
    ## export result
    return pop

def read_refine_pop_data():
    pop_data = read_pop_data()
    pop_data = refine_pop_data(pop = pop_data.copy())
    return pop_data
    

## execute code
pop_data = build_or_cache(function = read_refine_pop_data,
    address = 'B_Intermediate/pop_data.csv.gz',
    permit = set_acceleration['pop_cache'])
log_time('RD2')

Build/Cache Decision: Cache


#### GD3 / RD3 - read and refine text data

In [10]:
def read_explanatory_text(addr = 'A_Input/explanation.docx', n = 100):
    explain = Document(addr).paragraphs
    explain = [txt_wrap(i.text, n) for i in explain]
    explain = '\n'.join(explain)
    return explain

## execute code
explanatory_text = read_explanatory_text()
log_time('RD3')

#### GD4 - load CAN/MEX shapefiles for use as masking layers

In [11]:
def read_shp(addr):
    shp = shapereader.Reader(addr).records()
    shp = next(shp).geometry
    return ShapelyFeature(shp.geoms, PlateCarree())

can_mask   = read_shp('A_Input/gadm40_CAN_shp/gadm40_CAN_0.shp')
mex_mask   = read_shp('A_Input/gadm40_MEX_shp/gadm40_MEX_0.shp')
usa_border = read_shp('A_Input/gadm40_USA_shp/gadm40_USA_0.shp')

#### RD4 - Reconcile geographic and population dataset tracts

In [12]:
def reconcile_data(geo = geo_data, pop = pop_data):
    pop = pop.loc[geo.index]
    return geo, pop

##  execute code
geo_data, pop_data = reconcile_data()

# Model Data

#### MD1 - Precalculate tract-to-tract distance matrix

In [13]:
## define function to do distance compuations in parallel
def measure_distance_in_parallel(geo = geo_data):
    
    xy = list(zip(geo.INTPTLAT.values, geo.INTPTLON.values))
    the_iter = list(range(0, len(xy)))
    
    ## define engine function that will run on each parallel process
    def measure_distance_parallel_slice(n, xy_col = xy):
        from geopy.distance import geodesic # for parallel process
        xy_col = xy_col.copy()
        xy_row = xy_col[n]
        xy_dist = []
        for i in xy_col[0:n]: xy_dist.append(0)
        for i in xy_col[n::]:
            xy_dist.append(int(round(geodesic(xy_row, i).miles)))
        return xy_dist

    ## run engine in parallel for each slice of the data
    with Cluster(n = set_acceleration['distance_parallel_cores']) as clust:
        view = clust.load_balanced_view()
        asyncresult = view.map_async(measure_distance_parallel_slice, the_iter)
        asyncresult.wait_interactive()
        result = asyncresult.get()
        
    ## package results and export
    result = np.array(result)
    result = result + result.T
    result = pd.DataFrame(result)
    result.index, result.columns = (geo.index, geo.index)
    return result

## execute code
tract_distance = build_or_cache(
    function = measure_distance_in_parallel,
    address = 'B_Intermediate/tract_distance.csv.gz',
    permit = set_acceleration['distance_cache'])
log_time('MD1')

Build/Cache Decision: Cache


#### MD2 - Make one-stage agglomeration clustering functions

In [14]:
%%capture

## score model
def agglom_score(clusters, dist):
    
    ## load libraries (enables parallel processing)
    import numpy as np
    
    ## construct matrix of points that are in the same group
    score = clusters.reshape(clusters.shape[0], 1)
    score = (score == score.T).astype(int)
    
    ## sum distance between points in the same cluster
    score = (dist * score).sum().sum()
    return score.astype(int)

## generate partition of census tracts into k clusters based on proximity
def agglom_ml(k, dist, score_func = agglom_score):
    
    ## load libraries (enables parallel processing)
    from sklearn.cluster import AgglomerativeClustering # for parallel process
    from numpy           import append
    
    ## divide census tracts into clusters
    ml = AgglomerativeClustering(n_clusters = k,
                                 affinity = 'precomputed',
                                 linkage = 'average',
                                 compute_full_tree = False)
    ml_clusters = ml.fit_predict(dist)
    
    ## score the quality of the cluster solution
    ml_score = score_func(ml_clusters, dist)
    ml_clusters = append(ml_clusters, ml_score)
    
    ## export results
    return ml_clusters

## find best solution (using the fit curve 'elbow' approach)
def find_best_cluster_solution(cluster_batch):

    ## extract x and y
    x = cluster_batch.columns.values
    y = cluster_batch.iloc[cluster_batch.shape[0] - 1, :].values
    
    ## regularize x and y
    x = (x - np.min(x)) / (np.max(x) - np.min(x))
    y = (y - np.min(y)) / (np.max(y) - np.min(y))
    x = x.reshape(-1, 1)
    y = y.reshape(-1, 1)
    
    ## generate reference x and y
    x_ref = np.arange(0, 1, 0.01)
    x_ref = x_ref.reshape(-1, 1)
    y_ref = 1 - x_ref
    x_ref = x_ref
    
    ## calculate distances between xy and xy_ref
    x_dist = (x - x_ref.T)**2
    y_dist = (y - y_ref.T)**2
    elbow_score = x_dist + y_dist
    del x_dist, y_dist, x, y, x_ref, y_ref
    
    ## find the solution that is furthest from the line of equality (the elbow)
    elbow_score = elbow_score.min(axis = 1)
    elbow_score = (elbow_score == np.max(elbow_score)).astype(int)
    
    ## package and export
    elbow_score = pd.DataFrame({'Elbow':elbow_score}).T
    elbow_score.columns = cluster_batch.columns
    cluster_batch = pd.concat([cluster_batch, elbow_score], axis = 0)
    
    ##  return object
    return cluster_batch
    
## fit clusters in parallel for k = 2 through 50
def agglom_batch(k_min = 2, k_max = 48, f = agglom_ml,
                 dist_mat = tract_distance,
                 cores = set_acceleration['cluster_parallel_cores']):
    
    ## bound check parameters and construct model list
    k_min = max(k_min, 2)
    k_max = max(k_min + 1, k_max) + 1
    k_range = list(range(k_min, k_max))
    
    ## set distance matrix default
    from functools import partial
    f = partial(f, dist = dist_mat)
    
    ## run models in parallel
    with Cluster(n = cores) as clust:
        view = clust.load_balanced_view()
        asyncresult = view.map_async(f, k_range)
        asyncresult.wait_interactive()
        result = asyncresult.get()
    
    ## package results and identity best solution
    result = pd.DataFrame(np.array(result).T)
    result.columns = k_range
    result.index = np.append(dist_mat.index, 'Fit')
    result = find_best_cluster_solution(result)
    result = pd.DataFrame(result, columns = k_range)
    return result

## test code
cluster_level_one = build_or_cache(
    function = agglom_batch,
    address = 'B_Intermediate/cluster_level_one.csv.gz',
    permit = set_acceleration['cluster_L1_cache'])
log_time('MD2')

#### MD3 - Conduct two-stage agglomeration clustering

In [15]:
%%capture

def extract_cluster_solution(cl1):
    i = cl1.loc['Elbow', ].astype(bool)
    cl1 = cl1.loc[ ~cl1.index.isin(['Fit', 'Elbow']), i].squeeze()
    return cl1

def agglom_l2(dist = tract_distance, func = agglom_batch,
              cl1 = cluster_level_one, ecs = extract_cluster_solution):
    
    ## extract level one partition and set max number of level two clusters
    cl1 = ecs(cl1)
    ## max_cl2_clusters = int(48 / max(cl1)) * 2 ## alternative approach
    max_cl2_clusters = max(cl1)
    
    ## generate level two clusters for the cities in each level one cluster
    cl2 = dict()
    for i in set(cl1):
        
        ## calculate 2nd level cluster solutions for each cluster
        cluster_tracts = cl1[cl1 == i].index
        dist_iter = dist.loc[cluster_tracts, cluster_tracts].copy()
        k_max_iter = min(max_cl2_clusters, dist.shape[0])
        cl2[i] = func(dist_mat = dist_iter, k_max = k_max_iter)
        
        ## define useful indexes
        only_cluster = ~cl2[i].index.isin(['Fit', 'Elbow'])
        best_elbow = cl2[i].loc['Elbow'] == 1
        
        ## extract elbow statistics
        #elbow_range = best_elbow.copy()
        #elbow_range.iat[0] = True
        #elbow_range.iat[elbow_range.shape[0]-1] = True
        stats = [cl2[i].loc[~only_cluster, ]]
        stat_container = [np.nan for k in range(0, cl2[i].shape[0]-2)]
        stat_container[0] = stats

        ## extract the best L2 cluster assignments for each L1 cluster
        cl2[i] = cl2[i].loc[only_cluster, best_elbow]
        cl2[i].columns = ['l2_cluster']
        cl2[i]['l1_cluster'] = int(i)
        cl2[i]['l2_stats'] = stat_container
        
    ## consolidate and export data
    cl2 = pd.concat(cl2.values(), axis = 0).reset_index()
    cl2 = cl2.rename({'index':'GEO_ID'}, axis = 1).set_index('GEO_ID')
    return cl2

## execute code (Note - stats object is too complex to reassemble correctly.
   ## This ok for now as it is only being saved for 'just-in-case' reasons.
   ## If needed, it can be converted to a csv-format text string
cluster_level_two = build_or_cache(
    function = agglom_l2,
    address  = 'B_Intermediate/cluster_level_two.csv.gz',
    permit   = set_acceleration['cluster_L2_cache']
    )
geo_data = geo_data.join(cluster_level_two)
del cluster_level_two
geo_data['l2_index'] = geo_data.l1_cluster.astype(str) +\
    (geo_data.l2_cluster + 65).apply(chr)
log_time('MD3')

# Calculate Cluster Statistics

#### CSS1 - Aggregate statistics to cluster level

In [16]:
## compile data
cluster_data = pop_data.join(geo_data[['INTPTLON', 'INTPTLAT', 'l2_index']])
cluster_data = cluster_data.set_index('l2_index')
cluster_data = cluster_data.drop(['DP05_0077E'], axis = 1)

## transform education into pseudo years of education variable
cluster_data['edu_years'] = (cluster_data['DP02_0068E'] * 16)
cluster_data['edu_years'] = cluster_data['edu_years'] + (
    (cluster_data['DP05_0001E'] - cluster_data['DP02_0068E']) * 12)
cluster_data['edu_years'] = (
    cluster_data['edu_years'] / cluster_data['DP05_0001E'])

## calculate weighted averages and population count sums
column_operation = {
    'DP02_0068E': 'sum', 'DP03_0088E': 'mean', 'life_expect': 'mean',
    'rep_vote': 'sum', 'total_vote': 'sum', 'edu_years': 'mean',
    'INTPTLON': 'mean', 'INTPTLAT': 'mean'}

for i in column_operation.keys():
    if column_operation[i] == 'mean':
        cluster_data[i] = cluster_data[i] * cluster_data['DP05_0001E']
        
cluster_data = cluster_data.groupby('l2_index').sum()

for i in column_operation.keys():
    if column_operation[i] == 'mean':
        cluster_data[i] = cluster_data[i] / cluster_data['DP05_0001E']

        
## inflate counts to compensate for sampling
for i in column_operation.keys():
    if column_operation[i] == 'sum':
        cluster_data[i] = cluster_data[i] / set_acceleration['sample_size']
cluster_data['DP05_0001E'] = ((cluster_data['DP05_0001E']
                             ) / set_acceleration['sample_size']).astype(int)

## scale to hdi factors
cluster_data['hdi_edu'] = ((cluster_data['edu_years'] / 15) + (
    cluster_data['edu_years'] / 18)) / 2
cluster_data['hdi_inc'] = (np.log(cluster_data['DP03_0088E']) - np.log(100)
    ) / (np.log(75e3) - np.log(100))
cluster_data['hdi_lif'] = (cluster_data['life_expect'] - 20) / (85 - 20)

## calculate hdi
cluster_data['hdi'] = (
    cluster_data['hdi_edu'] * cluster_data['hdi_inc'] * cluster_data['hdi_edu'])
cluster_data['hdi'] = ((cluster_data['hdi'] ** (1/3)) * 1000).astype(int)
cluster_data['hdi_label'] = cluster_data['hdi'].astype(str)

## clean up columns
cluster_data = cluster_data.rename({
    'DP02_0068E':'education', 'DP03_0088E':'income',
    'DP05_0001E':'population'}, axis = 1)
cluster_data = cluster_data.drop(
    ['hdi_edu', 'hdi_inc', 'hdi_lif', 'education'], axis = 1)
cluster_data['income'] = cluster_data['income'].round(0).astype(int)

## 
cluster_data['rep_pct'] = cluster_data['rep_vote'] / cluster_data['total_vote']
cluster_data['log_pop'] = np.log(cluster_data['population'])

#### CSS2 - Fill in explanation statistics

In [17]:
def calculate_explain_stats(txt = explanatory_text, dat = cluster_data):
    
    ## make shell to hold statistics
    stats = dict()
    
    ## calculate number of new states
    stats['state_count'] = dat.shape[0]
    
    ## median scores
    stats['hdi_median'] = dat.hdi.median().astype(int)
    stats['pop_median'] = (dat.population.median()/ 1e6).round(1)
    stats['vote_median'] = (dat.rep_pct.median() * 1e2).round(1).astype(int)
    
    ## Figure 3 statistics
    x = dat.copy().population.sort_values(ascending = False)
    stats['pop_top_sum'] = 0
    stats['pop_top_count'] = 0
    for i in x.index:
        stats['pop_top_sum'] += x[i]
        stats['pop_top_count'] += 1
        if stats['pop_top_sum'] > (x.sum() * (1/2)): break
    stats['pop_other_count'] = stats['state_count'] - stats['pop_top_count']
    stats['pop_top_sum'] = (stats['pop_top_sum'] / 1e6).round(1)
    del x
    
    ## Figure 4 statistics - politics
    x = dat.copy().rep_pct.sort_values(ascending = False) - 0.5
    stats['rep_count'] = (x >=  0.05).sum()
    stats['dem_count'] = (x <= -0.05).sum()
   
    
    
    ## Figure 4 statistics - politics x hdi
    x = pd.concat([x, dat['hdi']], axis = 1)
    x['rep'] = pd.cut(x.rep_pct, [-9999, -0.05, 0.05, 9999],
                  labels = ['dem', 'swing', 'rep'])
    x = x.groupby('rep').median().round(0).astype(int)
    stats['rep_lean_hdi'] = x.loc['rep', 'hdi']
    stats['dem_lean_hdi'] = x.loc['dem', 'hdi']
    
    ## insert statistics into the text
    return txt.format(**stats)

## execute code
explanatory_text = calculate_explain_stats()

# Prepare Visualization Data

#### PVD1 - Generate Tract Centroid Voronoi Decomp. Polygons

In [18]:
def calculate_voronoi(xy = geo_data[['INTPTLON', 'INTPTLAT']]):
    
    ## calculate voronoi decomposition
    vor_obj = Voronoi(np.array(xy))
    
    ## assemble polygons for each point
    vor_polys = []
    for i in vor_obj.point_region:
        j = [x for x in vor_obj.regions[i] if x >= 0]
        vor_polys.append(vor_obj.vertices[j, :])
            
    ## generate valid polygons and return object
    for i in range(0, len(vor_polys)):
        try: vor_polys[i] = make_valid(Polygon(vor_polys[i]))
        except: vor_polys[i] = None
    return vor_polys
        

## execute code
tract_polys = calculate_voronoi()
log_time('PVD1')

#### PVD2 - Generate Cluster Polygons

In [19]:
def merge_polys(polys, groups):

    ## sort polygons into cluster groups
    cluster_poly = dict()
    for i in set(groups.values): cluster_poly[i] = []
    for i in range(0, len(polys) - 1):
        group_name = groups.iat[i]
        if polys[i] is not None:
            cluster_poly[group_name].append(polys[i])

    ## merge cluster group polygons and return
    for i in cluster_poly.keys():
        cluster_poly[i] = unary_union(cluster_poly[i])

    return cluster_poly

## execute code
clust_polys_l1 = merge_polys(tract_polys, geo_data.l1_cluster)
clust_polys_l2 = merge_polys(tract_polys, geo_data.l2_index)
log_time('PVD2')

# Render Visualization

#### RV0 - make the basic infrastructure for the visualization

In [20]:
%%capture

def draw_plot_foundation():
    
    ## make figure and grid objects
    global poster_fig
    poster_fig = plt.figure(figsize = (36, 24))
    poster_grid  = poster_fig.add_gridspec(12, 18, figure = poster_fig,
                                        hspace = 0.01, wspace = 0.01, top = 1,
                                        left = 0, right = 1, bottom = 0)
    poster_fig.set_facecolor(set_color['OrangeDark'])
    
    ## --
    
    ## define axes
    global poster_ax
    poster_ax = dict()
    
    ## main map (6x9)
    poster_ax['map1'] = poster_fig.add_subplot(
        poster_grid[0:6, 0:9], projection = set_map['map_proj'])

    
    ## explanation panel (6x9)
    poster_ax['explain'] = poster_fig.add_subplot(poster_grid[6:12, 0:9])

    ## 2x1 bar charts (6x3)
    poster_ax['bar1'] = poster_fig.add_subplot(poster_grid[0:6, 9:12])
    poster_ax['bar2'] = poster_fig.add_subplot(poster_grid[0:6, 12:15])
    poster_ax['bar3'] = poster_fig.add_subplot(poster_grid[0:6, 15:18])
    
    ## 1x2 mini-maps (3x3)
    poster_ax['minimap1'] = poster_fig.add_subplot(
        poster_grid[6:8, 9:12], projection = set_map['map_proj'])
    poster_ax['minimap2'] = poster_fig.add_subplot(
        poster_grid[6:8, 12:15], projection = set_map['map_proj'])
    poster_ax['minimap3'] = poster_fig.add_subplot(
        poster_grid[6:8, 15:18], projection = set_map['map_proj'])
    
    ## methods
    poster_ax['elbow'] = poster_fig.add_subplot(poster_grid[8:10, 9:12])
    poster_ax['map2'] = poster_fig.add_subplot(
        poster_grid[10:12, 9:12], projection = set_map['map_proj'])
    poster_ax['method'] = poster_fig.add_subplot(poster_grid[8:12, 12:18])

    ## remove axis ticks
    for i in poster_ax.keys():
        poster_ax[i].tick_params(
            bottom = False, top = False, left = False, right = False,
            labelbottom = False, labeltop = False, labelleft = False,
            labelright = False, color = 'red')
        poster_ax[i].set_facecolor(set_color['OrangeBG'])
        
## execute code
draw_plot_foundation()
log_time('RV0')

#### RV1 - draw explanation panel

In [21]:
def render_explanation(txt = explanatory_text):
    poster_ax['explain'].set_ylim(0, 6)
    poster_ax['explain'].set_xlim(0, 9)
    poster_ax['explain'].text(x = 9 * 0.03, y = 6 * 0.03, s = txt,
        fontweight = 'bold',
        horizontalalignment = 'left', verticalalignment = 'bottom',
        fontsize = set_font['medium'], color = set_color['OrangeDark'])
    
render_explanation()

#### RV2 - draw cluster map

In [22]:
## -- layers
## 0 - States
## 1 - Lakes
## 3 - Tract centroids
## 5, 6 - Cluster boundaries
## 8 - Ocean
## 9 - Cluster labels

## draw generic map layers
def draw_generic_map(ax, map_mask = [can_mask, mex_mask]):
    poster_ax[ax].set_extent(set_map['bounds'])
    poster_ax[ax].add_geometries(list(feature.STATES.geometries()),
        facecolor = '#33221100', edgecolor = set_color['OrangeLight'],
        lw = 1, zorder = 0, crs = PlateCarree())
    poster_ax[ax].add_feature(feature.LAKES,
        facecolor = set_color['OrangeBG'], edgecolor = set_color['OrangeLight'],
        lw = 1, zorder = 1)
    for i in map_mask:
        poster_ax[ax].add_feature(i, facecolor = set_color['OrangeBG'],
            edgecolor = set_color['OrangeLight'], lw = 1, zorder = 7)
    poster_ax[ax].add_feature(feature.OCEAN,
        facecolor = set_color['OrangeBG'], edgecolor = set_color['OrangeLight'],
        lw = 1, zorder = 8)
    
## draw census tract centroids
def draw_census_centroids(ax, geo = geo_data):
    poster_ax[ax].scatter(
        x = geo_data.INTPTLON.values, y = geo_data.INTPTLAT.values,
        color = set_color['OrangeMedium'], zorder = 3,
        transform = PlateCarree(), s = 1)
    
## label clusters
def label_cluster(ax, label_col = None, dat = cluster_data):

    if label_col is None: dat['label'] = dat.index
    else: dat['label'] = dat.index + '\n' + dat[label_col]

    bb = dict(edgecolor = '#33221100', lw = 3,
              facecolor = set_color['AzureBG'] + '80', boxstyle = 'round')

    for i in dat.index:
        poster_ax[ax].text(
            x = dat.at[i, 'INTPTLON'], y = dat.at[i, 'INTPTLAT'],
            s = dat.at[i, 'label'],
            fontsize = set_font['small'], color = set_color['AzureDark'],
            transform = PlateCarree(), fontweight = 'bold',
            bbox = bb, zorder = 9)

## draw cluster boundaries
def draw_cluster(ax, poly, the_lw = 2, zo = 5):
    for i in poly.keys():
        poster_ax[ax].plot(*poly[i].exterior.xy,
            zorder = zo, color = set_color['AzureBG'],
            transform = PlateCarree(), lw = the_lw * 1.5)
        poster_ax[ax].plot(*poly[i].exterior.xy,
            zorder = zo + 1, color = set_color['AzureDark'],
            transform = PlateCarree(), lw = the_lw)

## execute code - map 1
draw_generic_map('map1')
draw_census_centroids('map1')
draw_cluster('map1', clust_polys_l2)
label_cluster('map1')

## execute code - map 1
draw_generic_map('map2')
draw_census_centroids('map2')
draw_cluster('map2', clust_polys_l1)

#### RV3 - bar panels

In [23]:
def draw_bar(ax, major_col, minor_col = None, dat = cluster_data.copy(),
             unit = None):
    
    dat = dat.sort_values('hdi').copy()
    
    ## scale data for visualization purposes
    def min_max_scale(x):
        x = (x - min(x)) / (max(x) - min(x))
        n = 0.99
        x = (x * n) + (1 - n)
        return x
    
    dat['scaled_major'] = min_max_scale(dat[major_col])
        
    ## convert bar labels
    dat['label_major'] = dat[major_col]
    if unit == '/M':
        dat['label_major'] = (dat['label_major'] / 1e6).round(1)
        dat['label_major'] = dat['label_major'].astype(str) + 'M'
        min_display_major = 0
    elif unit == '%':
        dat['label_major'] = (dat['label_major'] * 1e2).round(0).astype(int)
        dat['label_major'] = dat['label_major'].astype(str) + '%'
        min_display_major = 0
    else:
        min_display_major = 0
          
    ## calculate display parameters
    bar_ceiling = 0.75
    bar_start = 0.10
    lab_pad = 0.02
    poster_ax[ax].set_xlim(0, 1)
    poster_ax[ax].set_ylim(0, 2)
    bar_space = 1.9 / (dat.shape[0] + 1)
    dat['bar_spacing'] = np.arange(bar_space, 1.9, bar_space)
    bb_blue = dict(edgecolor = set_color['AzureDark'], lw = 2,
             facecolor = set_color['AzureBG'] + '80', boxstyle = 'square')
    bb_orange = dict(edgecolor = set_color['OrangeMedium'], lw = 2,
             facecolor = set_color['OrangeLight'], boxstyle = 'square')

    ## draw major bar
    poster_ax[ax].barh(
        y = dat['bar_spacing'], height = bar_space,
        width = dat['scaled_major'] * bar_ceiling,
        color = set_color['OrangeLight'], edgecolor = set_color['OrangeDark'],
        lw = 2, left = bar_start)
    
    ## draw bar labels
    for i in dat.index:
        poster_ax[ax].text(
            y = dat.loc[i, 'bar_spacing'], x = lab_pad, s = i,
            horizontalalignment = 'left', verticalalignment = 'center',
            fontsize = set_font['small'], fontweight = 'bold',
            color = set_color['OrangeDark'])
        
    ## label major bars
    for i in dat.index:
        if dat.loc[i, major_col] > min_display_major:
            poster_ax[ax].text(
                y = dat.loc[i, 'bar_spacing'],
                x = bar_start + dat.loc[i, 'scaled_major'] * (
                    bar_ceiling + 0.01),
                s = dat.loc[i, 'label_major'],
                horizontalalignment = 'left', verticalalignment = 'center',
                fontsize = set_font['small'], fontweight = 'bold',
                color = set_color['OrangeDark'])

## execute code
draw_bar('bar1', 'hdi')
draw_bar('bar2', 'population',unit = '/M')
draw_bar('bar3', 'rep_pct', unit = '%')

#### RV4 - Simplified Color Maps

In [24]:
def brew_colors(stat_series, stat_cuts):
    stat_cuts = np.append(stat_cuts.values, 9e9)
    stat_cuts = np.append(-9e9, stat_cuts)
    stat_labels = ['AzureBG', 'AzureLight', 'AzureMedium']
    stat_series = pd.cut(stat_series, stat_cuts, labels = stat_labels)
    stat_series = stat_series.replace(set_color)
    return stat_series

def draw_color_cluster(ax, poly_color = None,
        poly = clust_polys_l2, map_mask = [can_mask, mex_mask]):

    poster_ax[ax].set_extent(set_map['bounds'])
    
    if poly_color is None:
        temp = dict()
        for i in poly.keys():
            temp[i] = '#FF0044'
        poly_color = temp

    for i in poly.keys():
        poster_ax[ax].fill(*poly[i].exterior.xy,
            zorder = 0, edgecolor = set_color['OrangeMedium'],
            facecolor = poly_color[i],
            transform = PlateCarree(), lw = 2)

    for i in map_mask:
        poster_ax[ax].add_feature(i, facecolor = set_color['OrangeBG'],
            edgecolor = set_color['OrangeLight'], lw = 1, zorder = 8)
    poster_ax[ax].add_feature(feature.OCEAN,
        facecolor = set_color['OrangeBG'], edgecolor = set_color['OrangeLight'],
        lw = 1, zorder = 9)

## execute code - color maps
new_state_colors = brew_colors(cluster_data['hdi'],
                         cluster_data.hdi.quantile([0.25, 0.75]))
draw_color_cluster('minimap1', new_state_colors)

new_state_colors = brew_colors(cluster_data['population'],
                         cluster_data.population.quantile([0.25, 0.75]))
draw_color_cluster('minimap2', new_state_colors)

new_state_colors = brew_colors(cluster_data['rep_pct'],
                         cluster_data.rep_pct.quantile([0.25, 0.75]))
draw_color_cluster('minimap3', new_state_colors)

#### RV5 - make titles

In [25]:
def make_title(ax, title):
    margin = max(poster_ax[ax].get_xlim()) * 0.01
    poster_ax[ax].text(
        x = max(poster_ax[ax].get_xlim()) - margin,
        y = max(poster_ax[ax].get_ylim()) - margin,
        s = title,
        fontsize = set_font['medium'], fontweight = 'bold',
        color = set_color['OrangeDark'], zorder = 11,
        horizontalalignment = 'right', verticalalignment = 'top')

## park titles here
make_title('map1', 'Figure 1: How State Borders Might Look If They ' +\
           'Corresponded To Where People Live')

make_title('bar1', 'Fig. 2a: HDI Score For Each Cluster')
make_title('bar2', 'Fig. 3a: Population Total For\nEach Cluster')
make_title('bar3', 'Fig. 4a: Percent Voting For The\nRepublican ' +\
           'Pres. Candidate in 2020')

make_title('minimap1', 'Figure 2b: HDI Quantile Map')
make_title('minimap2', 'Figure 3b: Population Quantile Map')
make_title('minimap3', 'Figure 4b: Republican Quantile Map')

make_title('method','Method: Calculating New State Borders')
make_title('elbow','Figure 5:')
make_title('map2', 'Figure 6:')

# Testing

#### Test 1 - 

#### Test 2 - 

#### Test 3 - 

# Footer

#### write poster to file

In [26]:
def save_poster():
    poster_fig.savefig('C_Output/pop_cluster_map.png')
    poster_fig.savefig('C_Output/pop_cluster_map.pdf')
save_poster()

In [27]:
log_time()

Time log:
   H2: 21:16:29
   H3: 21:16:29
  GD1: 21:16:29
  RD1: 21:16:29
  RD2: 21:16:29
  RD3: 21:16:29
  MD1: 21:17:34
  MD2: 21:17:34
  MD3: 21:17:34
 PVD1: 21:17:35
 PVD2: 21:17:36
  RV0: 21:17:36
  End: 21:19:14
